In [26]:
import psycopg2
import pandas as pd
import numpy as np

In [27]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="postgres"
)

In [28]:
cur = conn.cursor()

In [29]:
cur.execute("""
        select bank_id, transaction_type_chain, wight, swtc+swtv total
        from(
            select bank_id, transaction_type_chain, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                        from (select unnest(string_to_array(transaction_type_chain, ';')) transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                        where count = 0
                        group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
                        from (select unnest(string_to_array(transaction_type_chain, ';')) transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                        where count != 0) as b1) as b2
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
            group by bank_id, transaction_type_chain) as b4
            where bank_id = 13335
            order by total desc
            """)

In [30]:
result = cur.fetchall()
pd.DataFrame(result)

,0,1,2,3
0,13335,28392p,1219889,87522742252
1,13335,27333m,1074711,87506400317
2,13335,28391m,1085323,87506400317
3,13335,28691p,1166458,84918569263
4,13335,6s,1159733,67927028007
...,...,...,...,...
215,13335,27848c,5,445
216,13335,27848p,5,445
217,13335,27027p,5,385
218,13335,27811p,2,370


In [31]:
data = pd.read_csv('tranasaction_dataset.csv', parse_dates=['time_internal'])
data = data.replace(np.nan,0)
data['subchain'] = data['transaction_type_chain'].str.split(';')
data = data.drop(['transaction_type_chain'], axis='columns')
data = data.explode('subchain')

In [32]:
tmp_data = data[data['count'] == 0]
tmp_data
dg1 = tmp_data.groupby(by=['bank_id', 'subchain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [33]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'subchain', 'transaction_cost', 'transaction_value']]

In [34]:
dg = pd.concat([dg1, dg2])

In [35]:
dg = dg.groupby(by=['bank_id', 'subchain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [36]:
dg = dg.groupby(by=['bank_id', 'subchain']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [37]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [38]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,subchain,count,total_value
10951,13335,28392p,1219889,87522742252
10856,13335,27333m,1074711,87506400317
10950,13335,28391m,1085323,87506400317
10975,13335,28691p,1166458,84918569263
10984,13335,6s,1159733,67927028007
...,...,...,...,...
10912,13335,27848p,5,445
10911,13335,27848c,5,445
10811,13335,27027p,5,385
10908,13335,27811p,2,370
